### Finetuning gpt 2 for text to sql generation

In [1]:
#importing necessary libraries 
import os 
import torch
from datasets import load_dataset 
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging
)

from trl import SFTTrainer
from peft import LoraConfig

C:\Users\Hp\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load and save model 

In [2]:
path_to_model = os.path.join('model','gpt-2')

if not os.path.exists(path_to_model):
    !mkdir {path_to_model}

In [35]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(path_to_model)
model = AutoModelForCausalLM.from_pretrained(path_to_model)

In [36]:
tokenizer.pad_token = tokenizer.eos_token

In [4]:
#save the model to the path specified

# tokenizer.save_pretrained(path_to_model)

# model.save_pretrained(path_to_model)

### Custom gpt on the dataset

In [42]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe2 = pipeline("text-generation", model=model,tokenizer=tokenizer)

In [43]:
prompt = "write sql code for; What is the average, minimum, and maximum age of all singers from France?"
output = pipe2(prompt)

# Print the generated output
print(output)

[{'generated_text': 'write sql code for; What is the average, minimum, and maximum age of all singers from France?\n\nThe average, minimum, and maximum age of all singers from France. We can find out more about the results with the following test program'}]


### Loading and Structuring the Data

In [7]:
import json

train_file_path = 'data\extracted_train.json'
test_file_path = 'data\extracted_test.json'

#train
with open(train_file_path,'r') as file:
    train_data = json.load(file)
    
#test 
with open(test_file_path,'r') as file:
    test_data = json.load(file)

In [15]:
#convert into huggingface dataset format
from datasets import Dataset

train_questions = train_data['train_questions']
train_queries = train_data['train_queries']

#List all employees who joined after 2020 ### SELECT * FROM employees WHERE join_date > '2020-01-01';

combined_dataset = []
for i in range(len(train_queries)):
    combined_dataset.append('Text: ' + train_questions[i] + '### Geneated_sql_code:  ' + train_queries[i])
    
data_dict = {
    'text' : combined_dataset
}

train_dataset = Dataset.from_dict(data_dict)

train_dataset

Dataset({
    features: ['text'],
    num_rows: 7000
})

### Setting Training Args and Training 

In [17]:
output_dir = os.path.join('output_dir','result')

if not os.path.exists(output_dir):
    !mkdir {output_dir}

In [18]:
# Fine-tuned model name
new_model = "gpt2-finetuned"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# # Activate 4-bit precision base model loading
# use_4bit = True

# # Compute dtype for 4-bit base models
# bnb_4bit_compute_dtype = "float16"

# # Quantization type (fp4 or nf4)
# bnb_4bit_quant_type = "nf4"

# # Activate nested quantization for 4-bit base models (double quantization)
# use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = output_dir

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training it is actually for one core 
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for. It specifies number of batches of data it passes trhough before making any updation in weights
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping) avoid gradients getting too small keep them around 0.3
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "adamw_torch"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03 #Learning Rate Increase: During these 300 warmup steps, 
#the learning rate increases linearly from 0 to the initial learning rate you specified.

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

PEFT Configuration

In [19]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Setting Training Parameters

In [20]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

Set SFTTrainer Parameters

In [22]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map: 100%|██████████| 7000/7000 [00:02<00:00, 3225.41 examples/s]


Finally train the Model 

In [23]:
trainer.train()

  1%|▏         | 25/1750 [01:52<1:19:23,  2.76s/it]

{'loss': 4.0207, 'grad_norm': 0.22374030947685242, 'learning_rate': 9.433962264150944e-05, 'epoch': 0.01}


  3%|▎         | 50/1750 [02:36<37:47,  1.33s/it]  

{'loss': 5.2678, 'grad_norm': 0.4831840693950653, 'learning_rate': 0.00018867924528301889, 'epoch': 0.03}


  4%|▍         | 75/1750 [04:22<1:18:46,  2.82s/it]

...

100%|██████████| 1750/1750 [1:33:41<00:00,  1.16s/it]

{'loss': 1.7142, 'grad_norm': 0.576436460018158, 'learning_rate': 0.0, 'epoch': 1.0}


100%|██████████| 1750/1750 [1:33:41<00:00,  3.21s/it]

{'train_runtime': 5621.5117, 'train_samples_per_second': 1.245, 'train_steps_per_second': 0.311, 'train_loss': 1.8909064331054688, 'epoch': 1.0}


TrainOutput(global_step=1750, training_loss=1.8909064331054688, metrics={'train_runtime': 5621.5117, 'train_samples_per_second': 1.245, 'train_steps_per_second': 0.311, 'total_flos': 235513591160832.0, 'train_loss': 1.8909064331054688, 'epoch': 1.0})

In [24]:
trainer.model.save_pretrained(new_model)

Merge Finetune and base Model

In [29]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(path_to_model)

model2 = PeftModel.from_pretrained(base_model, new_model)
model2 = model2.merge_and_unload()

# Reload tokenizer to save it
tokenizer2 = AutoTokenizer.from_pretrained(path_to_model)
tokenizer2.pad_token = tokenizer.eos_token

In [39]:
pipe = pipeline("text-generation", model=model2,tokenizer=tokenizer2)

In [55]:
prompt = "text: What are the names of the states where at least 3 heads were born? "
output = pipe(prompt)

# Print the generated output
print(output)

[{'generated_text': 'text: What are the maximum and minimum budget of the departments? ### Geneated_sql_code:  SELECT MAX(MAX(MIN(WHEN FULL))  >  MAX(WHEN FULL ) ) FROM departments ORDER BY Min('}]


Push Model to Hugging face

In [31]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [32]:
tokenizer2 = AutoTokenizer.from_pretrained(path_to_model)
model2 = AutoModelForCausalLM.from_pretrained(path_to_model)

In [50]:
#List all employees who joined after 2020 ### SELECT * FROM employees WHERE join_date > '2020-01-01';
train_data['train_questions']

['How many heads of the departments are older than 56 ?',
 'List the name, born state and age of the heads of departments ordered by age.',
 'List the creation year, name and budget of each department.',
 'What are the maximum and minimum budget of the departments?',
 'What is the average number of employees of the departments whose rank is between 10 and 15?',
 'What are the names of the heads who are born outside the California state?',
 "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?",
 'What are the names of the states where at least 3 heads were born?',
 'In which year were most departments established?',
 "Show the name and number of employees for the departments managed by heads whose temporary acting value is 'Yes'?",
 'How many acting statuses are there?',
 'How many departments are led by heads who are not mentioned?',
 'What are the distinct ages of the heads who are acting?',
 "List the states where both the secretary